In [87]:
import numpy as np

def matrix_factorization (Y, X, W, K, steps, eta) : 
    step = 0
    while (step < steps) :
        for i in range(len(Y)): 
            for j in range(len(Y[0])) : 
                if(Y[i][j] > 0) :
                    eij = Y[i][j] - np.dot(X[i,:], W[:,j])
                    for k in range(K) :
                        X[i][k] += eta * (eij * W[k][j]) #update X
                        W[k][j] += eta * (eij * X[i][k]) # update W
        step += 1

    return np.dot(X, W)

Y = [
    [5, 3, 0, 1,3],
    [4, 0, 0, 1,0],
    [1, 1, 0, 5,2],
    [1, 0, 0, 4,1],
    [0, 1, 5, 4,3]]

m = len(Y) # matrix not shape, nên dùng len  ( các hàng của ma trận)
print('Số hàng của ma trận Y:', m)
n = len (Y[0]) # số cột của ma trận Y
print('Số cột của ma trận Y :', n)
K = 3 # chỉ số cột của matrix X, chỉ số hàng của ma trận W
X = numpy.random.rand(M, K); # items
W = numpy.random.rand(K, N); # users


rs = matrix_factorization (R, X, W, K, 3000, 0.1)
print('Y \n: ', Y)
print('Matrix predict:\n ', rs)

Y = [[(round(rs[i][j])) for j in range(0, len(rs[0]))] for i in range(0, len(rs))] #làm tròn số
# for i in
print(Y)

Số hàng của ma trận Y: 5
Số cột của ma trận Y : 5
Y 
:  [[5, 3, 0, 1, 3], [4, 0, 0, 1, 0], [1, 1, 0, 5, 2], [1, 0, 0, 4, 1], [0, 1, 5, 4, 3]]
Matrix predict:
  [[5.         3.         5.23055843 1.         3.        ]
 [4.         2.67371367 3.96918202 1.         2.20395651]
 [1.         1.         3.58320503 5.         2.        ]
 [1.         1.63243533 2.16085705 4.         1.        ]
 [2.22806751 1.         5.         4.         3.        ]]
[[5.0, 3.0, 5.0, 1.0, 3.0], [4.0, 3.0, 4.0, 1.0, 2.0], [1.0, 1.0, 4.0, 5.0, 2.0], [1.0, 2.0, 2.0, 4.0, 1.0], [2.0, 1.0, 5.0, 4.0, 3.0]]


In [55]:
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

class MF( object):
    
    def __init__(self, Y_data, K, lamda = 0.1, X_init = None, W_init = None, 
            learning_rate = 0.5, max_iter = 1000, print_every = 100, user_based = 1):
        self.Y_raw_data = Y_data
        self.K = K #   K: chỉ số cột của ma trận X, hoặc chỉ số hàng của ma trận W(not K neighbor )
        self.lamda = lamda #regularization
        self.learning_rate = learning_rate # eta
        self.print_every = print_every #print after each iter =100
        self.max_iter = max_iter # max of interation
        self.user_based = user_based # uuCF =1, item-itemCF =0
        # number of n_users, n_items, n_ratings : add 1 since id start from 0
        self.n_users = np.int(np.max(Y_data[:, 0])) + 1
        self.n_items = np.int(np.max(Y_data[:,1])) + 1
        self.n_ratings = Y_data.shape[0]
        
        if(X_init is None): #new
            self.X = np.random.randn(self.n_items, K)
        else: # or from saved data
            self.X = X_init
        if W_init is None:
            self.W = np.random.randn(K, self.n_users)
        else:
            self.W = W_init
        # normalized data, update later in normalized_Y function
        self.Y_data_n = self.Y_raw_data.copy() # Ybar == ma trận normalized
    def normalize_Y(self):
        if self.user_based: # user-user CF
            user_cols = 0
            item_cols = 1
            n_objects = self.n_users
        # if item-item CF: chỉ cần hoán đổi 2 cột 0,1 trong data
        else: # item-item CF
            user_cols = 1
            item_cols = 0
            n_objects = self.n_items
        users = self.Y_raw_data[:,user_cols]
        self.mu = np.zeros((n_objects,))
        for n in range(n_objects):
            # row indices of rating done by user n
            # since indices need to be integers, we need to convert
            ids = np.where(users == n)[0].astype(np.int32)
            # indices of all ratings associated with user n
            item_ids = self.Y_data_n[ids, item_cols]
            # and the corresponding ratings 
            ratings = self.Y_data_n[ids, 2]
            # take mean
            m = np.mean(ratings)
            if np.isnan(m):
                m =0
            self.mu[n] = m
            # normalized
            self.Y_data_n[ids, 2] = ratings - self.mu[n]
    # Tính hàm mất mất loss function
    def loss(self):
        L =0 #loss function
        for i in range(self.n_ratings):
            # n: users, m : items, rate : rating
            n,m, rate = int(self.Y_data_n[i,0]), int ( self.Y_data_n[i,1]), self.Y_data_n[i,2]
            L += 0.5*( rate - self.X[m,:].dot(self.W[:,n]))**2
        L /= self.n_ratings
        # regularization
        L  += 0.5*self.lamda*(np.linalg.norm(self.X) + np.linalg.norm(self.W))
        return L
    
    # Xác định các items được đánh giá bởi 1 user, và users đã đánh giá 1 item và các ratings tương ứng:
    def get_item_rated_by_u ( self, user_id):
        ids = np.where(self.Y_data_n[:,0] == user_id)[0]
        item_ids = self.Y_data_n[ids, 1].astype(np.int32) # as integer
        ratings = self.Y_data_n[ids, 2]
        return (item_ids, ratings)
    def get_user_who_rate_i(self, item_ids):
        ids = np.where(self.Y_data_n[:,1] == item_ids)[0]
        user_ids = self.Y_data_n[ids,0].astype(np.int32) # as integer
        ratings = self.Y_data_n[ids, 2]
        return ( user_ids, ratings)
    
    # CẬP NHẬT X,W
    def updateX(self):
        for m in range(self.n_items):
            user_ids, ratings = self.get_user_who_rate_i(m)
            Wm = self.W[:, user_ids]
            #gradient dx
            grad_xm = -1*(ratings - self.X[m,:].dot(Wm)).dot(Wm.T)/self.n_ratings + self.lamda*self.X[m,:]
            self.X[m,:]  -= self.learning_rate*grad_xm.reshape((self.K,))
    def updateW(self):
        for n in range(self.n_users):
            item_ids, ratings = self.get_item_rated_by_u(n)
            Xn = self.X[item_ids, :]
            #gradient dw
            grad_wn = -(Xn.T).dot((ratings - Xn.dot(self.W[:, n])))/self.n_ratings + self.lamda*self.W[:, n]
            self.W[:,n] -= self.learning_rate*grad_wn.reshape((self.K,))

    # function chính
    def fit(self):
        self.normalize_Y()
        for it in range(self.max_iter):
            self.updateX()
            self.updateW()
            if( (it+1) % self.print_every == 0):
                rmse_train = self.evaluate_RMSE(self.Y_raw_data)
                print('iteration:', it+1,'loss: ', self.loss(),'rmse: ', rmse_train)
    def pred(self, u, i):
        u = int(u)
        i = int(i)
        if self.user_based: #uuCF
            bias = self.mu[u]
        else : #item-item CF
            bias = self.mu[i]
        pred = self.X[i,:].dot(self.W[:,u]) + bias
        if pred <0:
            return 0
        if pred >5:
            return 5
        return pred
    def pred_for_user(self, user_id):
        """
        predict ratings one user give all unrated items
        """
        ids = np.where(self.Y_data_n[:,0] == user_id)[0]
        item_rated_by_u = self.Y_data_n[ids, 1].tolist()
        # 
        pred = self.X.dot(self.W[:, user_id]) + self.mu[user_id]
        predict_ratings =[]
        
        for i in range(self.n_items):
            if i not in item_rated_by_u:
                predict_ratings.append((i, pred[i]))
        return predict_ratings
    def evaluate_RMSE(self, rate_test):
        n_tests = rate_test.shape[0]
        SE = 0
        for n in range(n_tests):
            predict = self.pred(rate_test[n, 0], rate_test[n, 1])
            SE += ( predict - rate_test[n, 2])**2
        RMSE = np.sqrt(SE/n_tests)
        return RMSE
    

# Áp dụng lên cơ sở dữ liệu 100K

In [58]:
# ÁP DỤNG LÊN CƠ SỞ MOVIE 100K
# Kết quả dự đoán : User-user MF
r_cols =['user_ids','movie_ids','rating','time']
rating_base = pd.read_csv('ml-100k/ml-100k/ua.base', sep ='\t', names =r_cols)
rating_test = pd.read_csv('ml-100k/ml-100k/ua.test', sep ='\t', names =r_cols)

rate_train = rating_base.as_matrix()
rate_test  = rating_test.as_matrix()
# Vì các dữ liệu user, movie trong data đánh chỉ số từ 1 mà code thì from =0 
rate_train[:,:2] -=1
rate_test[:,:2]  -=1
print(rate_train)

rs = MF(rate_train, K = 10, lamda = .1, print_every = 10, learning_rate = 0.75, max_iter = 100, user_based = 1)
rs.fit() # Update X,W
# evaluate data test = RMSE
RMSE = rs.evaluate_RMSE(rate_test)
print( 'User-based MF:, RMSE = ', RMSE)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


[[        0         0         5 874965758]
 [        0         1         3 876893171]
 [        0         2         4 878542960]
 ...
 [      942      1187         3 888640250]
 [      942      1227         3 888640275]
 [      942      1329         3 888692465]]
iteration: 10 loss:  5.715785166335137 rmse:  1.218879894416563
iteration: 20 loss:  2.664998914126435 rmse:  1.0399023096171047
iteration: 30 loss:  1.3577276399557245 rmse:  1.0311557010133365
iteration: 40 loss:  0.7622480902575237 rmse:  1.0308648117775743
iteration: 50 loss:  0.48933431812749967 rmse:  1.0308737046368692
iteration: 60 loss:  0.3641832954621304 rmse:  1.0308789555563376
iteration: 70 loss:  0.3067899202290828 rmse:  1.0308802687679561
iteration: 80 loss:  0.2804698123541857 rmse:  1.0308805666766114
iteration: 90 loss:  0.26839968528442554 rmse:  1.0308806329171674
iteration: 100 loss:  0.2628644616317742 rmse:  1.0308806475592638
User-based MF:, RMSE =  1.0431349688681795


In [60]:
# Chuẩn hóa data dựa trên item
# Item- item MF
rs = MF (rate_train, K = 10, lamda = 0.1, print_every = 10, learning_rate = 0.75, max_iter = 100, user_based = 0)
rs.fit()
RMSE = rs.evaluate_RMSE(rate_test)
print('Item - item MF: RMSE :', RMSE)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


iteration: 10 loss:  5.617711067768782 rmse:  1.1799684104017114
iteration: 20 loss:  2.6155295531296705 rmse:  1.0064403119634762
iteration: 30 loss:  1.3240812145238305 rmse:  0.9972791387041281
iteration: 40 loss:  0.7356933783668834 rmse:  0.9968269635403769
iteration: 50 loss:  0.4660516073373314 rmse:  0.9967985961696211
iteration: 60 loss:  0.3424083915649652 rmse:  0.9967958409168649
iteration: 70 loss:  0.28570846372592645 rmse:  0.9967954580630609
iteration: 80 loss:  0.25970693913835197 rmse:  0.9967953977241638
iteration: 90 loss:  0.2477831025148291 rmse:  0.9967953883414381
iteration: 100 loss:  0.24231503865481183 rmse:  0.9967953870221634
Item - item MF: RMSE : 1.042652628248659


In [61]:
# Chúng ta làm thêm 1 ví dụ nữa không có regularization ( lamda =0)
rs = MF (rate_train, K = 10, lamda = 0, print_every = 10, learning_rate = 0.75, max_iter = 100, user_based = 0)
rs.fit()
RMSE = rs.evaluate_RMSE(rate_test)
print('Item - item MF: RMSE :', RMSE)

iteration: 10 loss:  5.031383964751958 rmse:  2.1267168001027748
iteration: 20 loss:  4.7519570204697805 rmse:  2.1060688504764955
iteration: 30 loss:  4.498454141524314 rmse:  2.085713023274088
iteration: 40 loss:  4.267638595989777 rmse:  2.0656817639365803
iteration: 50 loss:  4.056775159757976 rmse:  2.045975075658997
iteration: 60 loss:  3.863538145138154 rmse:  2.026641764979188
iteration: 70 loss:  3.68593877600318 rmse:  2.0077296106903533
iteration: 80 loss:  3.5222673453730255 rmse:  1.9892108136077953
iteration: 90 loss:  3.3710467785467007 rmse:  1.9711003601555046
iteration: 100 loss:  3.2309950760964226 rmse:  1.953383032002398
Item - item MF: RMSE : 2.0344147207625833


In [68]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('ml-1m/ml-1m/ratings.dat', sep='::', names=r_cols, encoding='latin-1')
ratings = ratings_base.as_matrix()

# indices in Python start from 0
ratings[:, :2] -= 1
from sklearn.model_selection import train_test_split

rate_train, rate_test = train_test_split(ratings, test_size=0.33, random_state=42)
print (rate_train.shape, rate_test.shape)

rs = MF(rate_train, K = 2, lamda = 0.1, print_every = 2, learning_rate = 2, max_iter = 10, user_based = 0)
rs.fit()
# evaluate on test data
RMSE = rs.evaluate_RMSE(rate_test)
print ('\nItem-based MF, RMSE =', RMSE)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


(670140, 4) (330069, 4)
iteration: 2 loss:  6.760367386321009 rmse:  1.1129588600613776
iteration: 4 loss:  4.32825162762031 rmse:  1.0002375657461706
iteration: 6 loss:  2.8363956769365135 rmse:  0.9778054239843604
iteration: 8 loss:  1.8924622233718082 rmse:  0.9739974338985337
iteration: 10 loss:  1.2901718102749564 rmse:  0.9733750464995046

Item-based MF, RMSE = 0.9816557197388072
